In [ ]:
################################################################################################################
################################################## INSTALL MODULES #############################################
################################################################################################################

!pip install click | grep -v 'already satisfied'
!pip install numpy | grep -v 'already satisfied'
!pip install requests | grep -v 'already satisfied'
!pip install Pillow | grep -v 'already satisfied'
!pip install torch torchvision torchaudio | grep -v 'already satisfied'
!pip install pandas | grep -v 'already satisfied'
!pip install scipy | grep -v 'already satisfied'
print("All dependencies installed")


In [ ]:
################################################################################################################
################################################### IMPORT MODULES #############################################
################################################################################################################

import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy
from stablehash_dataset import StableHashDataset
from torch.utils.data import DataLoader

print("All modules imported")

In [ ]:
################################################################################################################
############################################# SET ENVIRONMENT VARIABLES ########################################
################################################################################################################

# Set Nvidia Network model 

network_pkl = "/home/ubuntu/nvidia-model/stylegan3-r-ffhq-1024x1024.pkl"

# Load training dataset and create iterator
data_dir = "/volumes/1/output"
data_csv = "/home/ubuntu/stablehash-reconstruct/training_data.csv"
outdir  = "/volumes/1/output"

seeds = list(range(63))
truncation_psi = 0.75
noise_mode = "const"
translate = 0,0
rotate = 0
class_idx = None

train_data = StableHashDataset(data_dir, data_csv)
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
#stablehash, original_img = next(iter(train_dataloader))

print("All variables set")

In [ ]:
################################################################################################################
######################################### VERIFY THAT ALL IMAGE FILES EXIST ####################################
################################################################################################################

with open(data_csv, "r") as f:
    print("")
    next(f)
    count = 0
    for line in f:
        image_count = line.strip().split(";")[0]
        if not os.path.isfile(image_count):
            print(f"File {image_count} does not exist")
        count += 1
        
print("Scanned",count,"files")

In [ ]:
################################################################################################################
#################################################### RUN SCRIPT ################################################
################################################################################################################

def make_transform(translate: Tuple[float,float], angle: float):
    m = np.eye(3)
    s = np.sin(angle/360.0*np.pi*2)
    c = np.cos(angle/360.0*np.pi*2)
    m[0][0] = c
    m[0][1] = s
    m[0][2] = translate[0]
    m[1][0] = -s
    m[1][1] = c
    m[1][2] = translate[1]
    return m


def generate_images():
    
    # 1) Load pre-trained Generator
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore
    print("Network loaded")
    
    # 2) Define mapping and synthesis network (freeze)
    mapping_n = G.mapping
    synthesis_n = G.synthesis
    for mapping_param in mapping_n.parameters(): # set mapping network params to trainable
        mapping_param.requires_grad = True
    for synthesis_param in synthesis_n.parameters(): # set synthesis params to frozen
        synthesis_param.requires_grad = False
    print("Defined boolean mapping")
    
    # 3) Define optimizer --> in this case we use ADAM
    optim = torch.optim.Adam(mapping_n.parameters(), lr=0.0025, betas=[0,0.99], eps=1e-8)
    print("Optimizer defined")
    
    # 4) Define simple MSE loss function
    mse_loss = torch.nn.MSELoss(reduction="mean")
    loss_value = 1001
    count = 0
    while loss_value > 1000:
  # for i in range(1000):
        count += 1
        stablehash, orig_img = next(iter(train_dataloader))
        w = G.mapping(torch.flip(stablehash, [1]), class_idx) # remove flip function --> only for testing
        gen_img = G.synthesis(w)
        gen_img = (gen_img * 127.5 + 128).clamp(0, 255).float()
        loss_value = mse_loss(gen_img, orig_img)
        loss_value.backward()
        optim.step() #gradient descent
        print(loss_value)
    if loss_value != loss_value:
        print("we failed. Count:")
        print(count)
        print(loss_value)
    else:
        print(gen_img)
        #z = np.random.RandomState(seed).randn(1, G.z_dim)
        #np.save(os.path.join(outdir, f"{seed}.npy"), z)
        #z = torch.from_numpy(z).to(device)
        #w = G.mapping(z, class_idx)
        #img = G.synthesis(w)
        #img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        #PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/{seed}.png')
        
        gen_img = gen_img.detach().cpu().numpy()
        np.save(f'{outdir}/test.npy', gen_img)
        PIL.Image.fromarray(gen_img[0], 'RGB').save(f'{outdir}/test.png')
        print("Export gen_img to file")
        
    print("Exit")
        
generate_images() 